In [ ]:
#1-1

import pandas as pd
import numpy as np
dat = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_1_1.csv')
#print(df.info())
#print(df.head(3))

dat.dropna(inplace=True)
up_to_70 = len(dat)*.7
sub_dat = dat.loc[:up_to_70, :]
#print(sub_dat)

result = sub_dat['PTRATIO'].quantile(.25)
print(result.round(2))


#1-2
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_1_2.csv')
#print(df.info())
#print(df.head())

cond1 = df['yearBuilt'].between(1991,2000)
cond2 = df['avgSchoolRating'] <= df['avgSchoolRating'].mean()
result1 = df[cond1 & cond2]['uid']
print(result1)

cond3 = df['yearBuilt'].between(2001,2010)
cond4 = df['avgSchoolRating'] >= df['avgSchoolRating'].mean()
result2 = len(df[cond3 & cond4]['uid'])
result3 = len(df[cond1 & cond2]['uid'])
print(result2 + result3)


#1-3
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_1_2.csv')
# print(df.info())
# print(df.head(15))

result3 = df.isna().sum().idxmax()
print(result3)

16.32
24      6815
28      8710
40      6191
54     12148
71      3854
85     12086
89     10308
152     9088
190     6741
222     2747
267     5241
278     7686
299    10345
335     5054
386     6166
415    12198
438     8437
448    12449
456     7698
459     8766
477     5708
Name: uid, dtype: int64
76
numOfPatioAndPorchFeatures


In [165]:
#2
import pandas as pd
import numpy as np
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_2_test.csv')

# 2-1 (인포)
print(train.info())
print(test.info())

# 2-2 (독-종 분리)
train_X = train.drop('Attrition_Flag', axis=1)
train_y = train['Attrition_Flag']
test_X = test.drop('Attrition_Flag', axis=1)
test_y = test['Attrition_Flag']
test_id = test['ID']

# 2-3 (검증 데이터 생성)
from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.3, random_state=1)

# 2-4 (cat-num 분리)
cat_columns = train_X.select_dtypes('object').columns.to_list()
num_columns = train_X.select_dtypes('number').columns.to_list()

# 2-5 (전처리)
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
stdscaler = StandardScaler()

train_X_oe = oe.fit_transform(train_X[cat_columns])
valid_X_oe = oe.transform(valid_X[cat_columns])
test_X_oe = oe.transform(test_X[cat_columns])

train_X_std = stdscaler.fit_transform(train_X[num_columns])
valid_X_std = stdscaler.transform(valid_X[num_columns])
test_X_std = stdscaler.transform(test_X[num_columns])

train_X_preprocessed = np.concatenate([train_X_oe, train_X_std], axis=1)
valid_X_preprocessed = np.concatenate([valid_X_oe, valid_X_std], axis=1)
test_X_preprocessed = np.concatenate([test_X_oe, test_X_std], axis=1)

# 2-6 (모델 적합)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1)
rf.fit(train_X_preprocessed, train_y)

# 2-7 (검증, 성능확인)
from sklearn.metrics import roc_auc_score
pred_val = rf.predict_proba(valid_X_preprocessed)[:,1]
print(roc_auc_score(valid_y, pred_val))

# 2-8 (최종예측)
test_pred = rf.predict_proba(test_X_preprocessed)[:,1]
test_pred = pd.DataFrame(test_pred, columns = ['prob'])

# 2-9 (생성 저장)
result = pd.concat([test_id, test_pred], axis=1)
result.to_csv('result.csv', index=False)
print(pd.read_csv('result.csv').head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      354 non-null    int64  
 1   Attrition_Flag          354 non-null    object 
 2   Gender                  354 non-null    object 
 3   Customer_Age            354 non-null    int64  
 4   Income_Category         354 non-null    object 
 5   Card_Category           354 non-null    object 
 6   Credit_Limit            354 non-null    float64
 7   Total_Revolving_Bal     354 non-null    int64  
 8   Avg_Utilization_Ratio   354 non-null    float64
 9   Avg_Open_To_Buy         354 non-null    float64
 10  Months_Inactive_12_mon  354 non-null    int64  
dtypes: float64(3), int64(4), object(4)
memory usage: 30.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 11 columns):
 #   Column                  Non-

In [ ]:

#3-1
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_3_1.csv')
#print(df.info())
#print(df.head())

#3-1-1
A = df.loc[df['department'] == 'A', :]
B = df.loc[df['department'] == 'B', :]
A_mean = round(A['hours_worked'].mean(), 2)
A_std = round(A['hours_worked'].std(), 2)
B_mean = round(B['hours_worked'].mean(), 2)
B_std = round(B['hours_worked'].std(), 2)

print(A_mean, A_std, B_mean, B_std)

#3-1-2
from scipy.stats import ttest_ind
statistic, p_value = ttest_ind(A['hours_worked'], B['hours_worked'])
print(round(statistic, 2))

#3-1-3
pvalue = p_value.round(2)

if pvalue < 0.05:
  result = '기각'
else:
  result = '채택'

print(result)

#3-2
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_3_2.csv')
#print(df.info())
#print(df.head())

#3-2-1
A = df.loc[df['factory'] == 'A', :]
B = df.loc[df['factory'] == 'B', :]
C = df.loc[df['factory'] == 'C', :]
A_mean = round(A['quality_score'].mean(), 2)
A_std = round(A['quality_score'].std(), 2)

B_mean = round(B['quality_score'].mean(), 2)
B_std = round(B['quality_score'].std(), 2)

C_mean = round(C['quality_score'].mean(), 2)
C_std = round(C['quality_score'].std(), 2)

print(A_mean, A_std, B_mean, B_std, C_mean, C_std)

#3-2-2
from scipy.stats import kruskal

statistic, p_value = kruskal(A['quality_score'],B['quality_score'],C['quality_score'])

print(statistic.round(2))

#3-2-3
p_value = p_value.round(2)

if p_value < 0.05:
  result = '기각'
else:
  result = '채택'

print(result)

42.21 5.5 43.55 4.57
-1.02
채택
74.43 11.0 72.1 9.14 78.66 9.65
5.93
채택
